# PRE-PROCESSING

Ở bước này, sau khi hoàn thành bước làm sạch (những bước xử lý ban đầu để dữ liệu trực quan hóa được) và khám phá dữ liệu, em sẽ tận dụng những kết quả phân tích đó, kết hợp với một chút suy luận để đưa ra những features mới, xử lý dữ liệu sau cho phù hợp với quá trình **Modeling**.

Lưu ý: dữ liệu xử lý được ở file này sẽ dùng cho mục đích **Modeling** nên em sẽ thực hiện một số thay đổi có thể ảnh hưởng đến dữ liệu ban đầu (log-scale, bỏ những features không phù hợp cho quá trình **Modeling**,...). Tuy nhiên, do Model được sử dụng sẽ là Model Regression cơ bản nên bước **Pre-processing** cũng chỉ thực hiện những bước phù hợp với Model.

## I. IMPORT THƯ VIỆN

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sys
sys.path.append("..")

import src

### PHIÊN BẢN NUMPY SỬ DỤNG

In [2]:
np.set_printoptions(suppress=True, precision=6, linewidth=150)
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("Numpy version:", np.__version__)

Numpy version: 2.2.3


## II. LOAD DATA

Quan sát ban đầu về dữ liệu, chiến lược thu thập, hạn chế, giải pháp

### 1. Đọc file

In [3]:
data, columns = src.load_data("../data/processed/AB_NYC_2019_exploration.csv")

Loaded: (48895, 16)


In [4]:
src.print_preview(data, columns, n=5, col_width=25)

id                        | name                      | host_id                   | host_name                 | neighbourhood_group       | neighbourhood             | latitude                  | longitude                 | room_type                 | price                     | minimum_nights            | number_of_reviews         | last_review               | reviews_per_month         | calculated_host_listings_ | availability_365          | 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2539                      | Clean & quiet apt home... | 2787                      | John              

### 2. Tách numerical và categorical

In [5]:
id_col_idx = [0, 2]   
date_col_idx = [12]   
cat_cols_idx = [1, 3, 4, 5, 8] 
num_cols_idx = [6, 7, 9, 10, 11, 13, 14, 15]

id_data = data[:, id_col_idx]
cat_data = data[:, cat_cols_idx]

# cái này tách thui chứ em sẽ không xử lý thêm
date_data = data[:, date_col_idx]

# bước này em làm cho khoa học, chuẩn quy trình thui chứ dữ liệu đã sạch rồi
temp_num = data[:, num_cols_idx]
temp_num[temp_num == ''] = 'nan'
num_data = temp_num.astype(float)

*Kiểm tra xem numeric có tồn tại giá trị `nan` không (cho chắc)*

In [6]:
nan_mask = np.isnan(num_data)
num_nans = np.sum(nan_mask, axis=0)
for i, col_idx in enumerate(num_cols_idx):
    if num_nans[i] > 0:
        print(f"Column '{columns[col_idx]}' has {num_nans[i]} NaN values.")

Vậy là không còn sót lại giá trị `nan` nào (Bước xử lý missing values ở `01_data_exploration.ipynb` đã làm tốt).

In [7]:
# Numerical columns
lat = num_data[:, 0]
long = num_data[:, 1]
price = num_data[:, 2]
min_nights = num_data[:, 3]
n_reviews = num_data[:, 4]
reviews_per_month = num_data[:, 5]
host_listings_count = num_data[:, 6]
availability_365 = num_data[:, 7]

# Categorical columns
name = cat_data[:, 0]
host_name = cat_data[:, 1]
neigh_group = cat_data[:, 2]
neighbourhood = cat_data[:, 3]
room_type = cat_data[:, 4]

## II. Feature Engineering

### 1. Tạo đặc trưng

Chọn tòa nhà Empire State Building làm trung tâm. Dùng 2 biến tọa độ (`lattitude` và `longtitude`) để tính khoảng cách từ điểm tới trung tâm.

In [8]:
# Tọa độ Empire State Building
CENTER_LAT = 40.7484
CENTER_LONG = -73.9857

In [9]:
dist_to_center = np.sqrt((lat - CENTER_LAT)**2 + (long - CENTER_LONG)**2)

dist_to_center = dist_to_center.reshape(-1, 1)

print(f"Feature 'Distance' created. Shape: {dist_to_center.shape}")
print(f"Min Dist: {dist_to_center.min():.4f}, Max Dist: {dist_to_center.max():.4f}")

Feature 'Distance' created. Shape: (48895, 1)
Min Dist: 0.0002, Max Dist: 0.3562


### 2. Xử lý các thuộc tính *Numerical*
- Xử lý ngoại lai.
- Chuẩn hóa.

Lưu ý: Ở đây em chỉ xử lý kĩ những feature em sẽ dùng cho quá trình **Modeling**, và chừa bước Standardization lại để làm sau khi chia tập train/test ở **Modeling**.

**Thêm các cột vừa tạo**

In [10]:
num_data = np.hstack([num_data, dist_to_center])

**Xử lý `price`**

*Loại bỏ giá trị = 0 và >= 500 (Dựa trên kết quar EDA)*

In [11]:
num_price_zero = np.sum(price == 0)
print(f"Số dòng có giá bằng 0: {num_price_zero}")

num_price_high = np.sum(price >= 500)
print(f"Số dòng có giá lớn hơn hoặc bằng 500: {num_price_high}")

Số dòng có giá bằng 0: 11
Số dòng có giá lớn hơn hoặc bằng 500: 1235


In [12]:
mask = (price > 0) & (price < 500)
num_data = num_data[mask]
cat_data = cat_data[mask]

*Log-scale `price`*

In [13]:
price_log = np.log1p(num_data[:, 2])
num_data[:, 2] = price_log

**Log-scale các thuộc tính dùng cho Modeling**

In [14]:
min_nights_log = np.log1p(num_data[:, 3])
num_data[:, 3] = min_nights_log

number_of_reviews_log = np.log1p(num_data[:, 4])
num_data[:, 4] = number_of_reviews_log

reviews_per_month_log = np.log1p(num_data[:, 5])
num_data[:, 5] = reviews_per_month_log

host_listings_count_log = np.log1p(num_data[:, 6])
num_data[:, 6] = host_listings_count_log

availability_365_log = np.log1p(num_data[:, 7])
num_data[:, 7] = availability_365_log

**Chọn biến dùng cho Modeling**

In [15]:
# Chọn các feature số cần đưa vào mô hình
# Bỏ lat, long (vì đã có distance), Bỏ price (là y)
# Features: [min_nights, n_reviews, rev_per_month, host_count, avail, distance]
num_data_modeling = num_data[:, [3, 4, 5, 6, 7, 8]]

### 3. Xử lý các thuộc tính *Categorical*


Tách các cột `cat_data` (Các cột kiểu text) ra thành 2 Mảng:
- `one_hot_data`: các thuộc tính được chọn để one-hot.
- `text_data`: giữ dạng text (Không dùng cho Modeling)

In [16]:
onehot_data = cat_data[:, [2, 4]]
text_data = cat_data[:, [0, 1, 3]]

One-hot dữ liệu

In [17]:
neigh_group_ohe, neigh_names = src.one_hot(onehot_data[:, 0])
print(f"Neighbourhood Groups encoded: {neigh_names}")

room_type_ohe, room_names = src.one_hot(onehot_data[:, 1])
print(f"Room Types encoded: {room_names}")

onehot_data = np.hstack([neigh_group_ohe, room_type_ohe])

Neighbourhood Groups encoded: ['Bronx' 'Brooklyn' 'Manhattan' 'Queens' 'Staten Island']
Room Types encoded: ['Entire home/apt' 'Private room' 'Shared room']


### Kết hợp dữ liệu, chia tập X/y

In [18]:
y = num_data[:, 2]

In [19]:
data_processed = np.hstack([num_data_modeling, onehot_data])

Thêm cột bias

In [20]:
intercept = np.ones((data_processed.shape[0], 1))
X = np.hstack([intercept, data_processed])

In [21]:
print("\n--- Final Dataset ---")
print(f"X shape: {X.shape} (Samples, Features)")
print(f"y shape: {y.shape}")


--- Final Dataset ---
X shape: (47649, 15) (Samples, Features)
y shape: (47649,)


### Đổi tên cột

In [22]:
feature_names = ["Intercept", "Minimum_nights", "number_of_reviews",
                  "reviews_per_month", "host_count", "availability_365",
                    "distance_to_center"]

feature_names.extend(neigh_names)
feature_names.extend(room_names)

## IV. Lưu dữ liệu

In [23]:
output_path = "../data/processed/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

file_path = f"{output_path}AB_NYC_2019_processed.npz"
np.savez(file_path, X=X, y=y)

print(f"Saved processed arrays to: {file_path}")

# Lưu tên feature để đối chiếu
np.save(f"{output_path}AB_NYC_2019_feature_names.npy", np.array(feature_names))
print("Saved feature names.")

Saved processed arrays to: ../data/processed/AB_NYC_2019_processed.npz
Saved feature names.
